<a href="https://colab.research.google.com/github/bsureshkrishna/neur503/blob/main/Neur503.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, convolve, find_peaks
from scipy.stats import gamma
import math
import sklearn
from numpy import real, pi
from scipy.fft import fft, ifft, fftfreq #Documentation: https://docs.scipy.org/doc/scipy/reference/fft.html#module-scipy.fft

def plot_signal_spectrum(signal, fs, plotornot = 1):
    # Compute the FFT
    fft_result = np.fft.fft(signal)

    # Compute the frequency bins
    freqs = np.fft.fftfreq(len(signal), d=1/fs)

    # Compute the magnitude (absolute value of the FFT)
    magnitude = np.abs(fft_result)

    if(plotornot==1):
        # Plotting
        sorted_indices = np.argsort(freqs)
        magnitude=magnitude[sorted_indices]
        freqs=freqs[sorted_indices]
        plt.plot(freqs, magnitude)
        plt.xlabel("Frequency (Hz)")
        plt.ylabel("Magnitude")
        plt.title('Spectrum')

    return freqs, magnitude

def amplitude_from_spl(spl):
    amplitude = np.power(10,(spl/20))*20*1e-6 #amplitude in pascals, since reference of 0dB = 20 micropascals
    return amplitude

def generate_tone(t,spl,frequency, phasevalue=0):
    #supply amplitdue in sound pressure level (decibels), frequency in hz
    amplitude = amplitude_from_spl(spl) #amplitude in micropascals, since reference of 0dB = 20 micropascals
    signal = amplitude*np.sin(2.0*np.pi*frequency*t+phasevalue) #a simple sine-wave to use as signal
    return signal

def nonlinearity(amplitudes, x0=10, Rmax=100, sigma=40):
    x0=-1.0*amplitude_from_spl(x0)
    if sigma<=0:
      sigma=sigma+0.1
    if Rmax<sigma:
      Rmax=sigma+10
    km=x0**2*((Rmax/sigma)-1) #km is defined in terms of sr.
    x = np.array(amplitudes)  # Ensure x is a NumPy array
    return np.where(x < x0, 0, Rmax * (x - x0)**2 / ((x - x0)**2 + km))

def timeplot(signal,fs,colorvar='blue'):
    plt.plot((1/fs)*np.arange(0,len(signal)),signal,color=colorvar)

def generate_gammatone(timeax,f=440,b=50,n=4,a=1,phi=0):
    gammapart=a*timeax**(n-1)*np.exp(-2.0*np.pi*b*timeax)
    gammapart=2*gammapart/np.trapz(gammapart)
    cospart=np.cos(2.0*np.pi*f*timeax) # a hack normalization to make convolution output "preserve" amplitude
    gammatone_impresp=cospart*gammapart #impulse response of gammatone filter
    return gammatone_impresp

def generate_samtone(t,spl,frequency, phasevalue=0, amfrequency=10):
    #supply amplitdue in sound pressure level (decibels), frequency in hz
    amplitude = amplitude_from_spl(spl) #amplitude in micropascals, since reference of 0dB = 20 micropascals
    signal = amplitude*(1+np.sin(2*np.pi*amfrequency*t-(np.pi)/2))*np.sin(2.0*np.pi*frequency*t+phasevalue) #a simple sine-wave to use as signal
    return signal

def print_vars(**kwargs):
    output = "; ".join([f"{name} = {value}" for name, value in kwargs.items()])
    print(output)

def myfft(signal, timeaxis, fs):
    #function calculates fft and ifft given signal
  signal_fft = (fft(signal, norm="forward")) #calculate forward (normalized) fft. in general, will be complex
  signal_ifft = real(ifft(signal_fft, norm="forward")) #invert the fft to hopefully recreate original signal. note that you cannot ignore phase !
  freqaxis = fftfreq(len(signal),d=1/fs) #frequency x-axis for fft
  return signal_fft, signal_ifft, freqaxis

def plotmyfft(signal, timeaxis, signal_fft, signal_ifft, freqaxis, fs):
    #function plots signal, fft and ifft
  fig,axs = plt.subplots(nrows = 3,
                         ncols = 1,
                         constrained_layout="true")

  ax=axs[0]
  ax.plot(timeaxis,signal) #plot signal
  ax.set_xlabel("Time (s)")
  ax.set_ylabel("Signal")
  ax.set_title("Signal")

  ax=axs[1]
  fft_abs=abs(signal_fft) #take the absolute value (aka magnitude) of the fft
  ax.plot(freqaxis,fft_abs) #plot fft magnitude. ignoring phase here
  peaks, _ = find_peaks(fft_abs, height=0.4) # You might need to adjust the height parameter based on your data
  ax.scatter(freqaxis[peaks], fft_abs[peaks], color='red') # mark peak points
  # Print peaks
  for peak in peaks:
    print_vars(peak=freqaxis[peak],magnitude=fft_abs[peak])
  ax.set_xlabel("Frequency (Hz)")
  #ax.set_xlim([-100, 100])
  ax.set_ylabel("Magnitude")
  ax.set_title("FFT of Signal")

  ax=axs[2]
  ax.plot(timeaxis,signal_ifft) #plot recovered time signal
  ax.set_xlabel("Time (s)")
  ax.set_ylabel("Recovered Signal")
  ax.set_title("Inverse FFT of Signal (Recovered signal)")

  return

def plotmyfft2(signal, timeaxis, signal_fft, freqaxis, fs):
    #function plots signal and fft.
  fig,axs = plt.subplots(nrows = 2,
                         ncols = 1,
                         constrained_layout="true")

  ax=axs[0]
  ax.plot(timeaxis,signal) #make plots
  ax.set_xlabel("Time (s)")
  ax.set_ylabel("Signal")
  ax.set_title("Signal")

  ax=axs[1]
  fft_abs=abs(signal_fft) #take the absolute value (aka magnitude) of the fft
  ax.plot(freqaxis,fft_abs) #plot fft magnitude. ignoring phase here
  peaks, _ = find_peaks(fft_abs, height=0.05) # You might need to adjust the height parameter based on your data
  ax.scatter(freqaxis[peaks], fft_abs[peaks], color='red') # mark peak points

  # Print peaks
  for peak in peaks:
    print_vars(peak=freqaxis[peak],magnitude=fft_abs[peak])

  ax.set_xlabel("Frequency (Hz)")
  # ax.set_xlim([-100, 100])
  ax.set_ylabel("Magnitude")
  ax.set_title("FFT of Signal")